In [ ]:
!pip install nerus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.9 MB/s eta 0:00:00


In [ ]:
from nerus import load_nerus
import sklearn_crfsuite
from sklearn_crfsuite import metrics as crf_metrics
#from ipymarkup import show_span_ascii_markup, show_dep_ascii_markup
#import ru_core_news_sm
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
import nltk
from sklearn.model_selection import train_test_split
import random
from collections import Counter
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Nerus

In [ ]:
!wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz

--2023-03-23 16:55:15--  https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1961465886 (1.8G) [application/octet-stream]
Saving to: ‘nerus_lenta.conllu.gz’

nerus_lenta.conllu. 100%[===================>]   1.83G  23.9MB/s    in 81s     

2023-03-23 16:56:37 (23.0 MB/s) - ‘nerus_lenta.conllu.gz’ saved [1961465886/1961465886]



In [ ]:
docs = load_nerus('/content/nerus_lenta.conllu.gz')

# Озон

In [ ]:
!wget dataset_ozon.tsv https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv

--2023-03-23 16:56:37--  http://dataset_ozon.tsv/
Resolving dataset_ozon.tsv (dataset_ozon.tsv)... failed: Name or service not known.
wget: unable to resolve host address ‘dataset_ozon.tsv’
--2023-03-23 16:56:37--  https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9640627 (9.2M) [text/plain]
Saving to: ‘dataset_ozon.tsv’

dataset_ozon.tsv    100%[===================>]   9.19M  --.-KB/s    in 0.03s   

2023-03-23 16:56:37 (263 MB/s) - ‘dataset_ozon.tsv’ saved [9640627/9640627]

FINISHED --2023-03-23 16:56:37--
Total wall clock time: 0.2s
Downloaded: 1 files, 9.2M in 0.03s (263 MB/s)


In [ ]:
ds = pd.read_csv('/content/dataset_ozon.tsv', sep = '\t')
ds = ds.drop(columns = ['Unnamed: 0'])

In [ ]:
ds

,name,brand
0,"Игровая консоль PlayStation 5, белый",PlayStation
1,Золотой Шелк Маска для волос гиалуроновая кисл...,Золотой Шелк
2,"Игровая консоль Microsoft Xbox Series X, черный",Microsoft
3,L'Oreal Paris Infaillible 24H Fresh Wear Пудра...,L'Oreal Paris
4,Maybelline New York Lifter Gloss Блеск для губ...,Maybelline New York
...,...,...
81774,"Erborian BB Family BB карандаш, светлый, 3 г",Erborian
81775,Комплект модулей сменных фильтрующих Аквафор Р...,Аквафор
81776,Набор менструальных чаш Satisfyer Feel secure ...,Satisfyer
81777,28048 Подстилка для надувных и каркасных бассе...,Intex


In [ ]:
def make_tags(n, b):
    global tags
    ner_tags_id = []
    name = wordpunct_tokenize(n)
    brand = wordpunct_tokenize(b)
    results=[]
    sll=len(brand)
    for ind in (i for i,e in enumerate(name) if e==brand[0]):
        if name[ind:ind+sll]==brand:
            results.append((ind,ind+sll-1))
    if results != []:
      ner_tags_labels = ['O'] * len(name)  
      ner_tags_id = [0] * len(name)     
      for result in results:
          if result[0] == result[1]:
              ner_tags_labels[result[0]] = 'B-BRAND'
              ner_tags_id[result[0]] = 1
          else:
            ner_tags_labels[result[0]] = 'B-BRAND'
            ner_tags_id[result[0]] = 1
            for j in range(result[0]+1, result[1]+1):
              ner_tags_labels[j] = 'I-BRAND'
              ner_tags_id[j] = 2

      tags.append({'tokens': name, 'ner_tags_labels': ner_tags_labels})

In [ ]:
#nlp = ru_core_news_sm.load()

In [ ]:
tags = []
ds.progress_apply(lambda x: make_tags(x['name'], x['brand']), axis = 1)

100%|██████████| 81779/81779 [00:03<00:00, 24955.56it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
81774    None
81775    None
81776    None
81777    None
81778    None
Length: 81779, dtype: object

# Формируем данные для CRF

In [ ]:
def find_in_nerus(docs, tag):
  global result
  count_tag = 0
  tags2id = {'B-PER':3, 'I-PER': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-ORG': 7, 'I-ORG': 8, 'O': 0}
  b_tag = 'B-' + tag
  i_tag = 'I-' + tag
  for doc in docs:
      if count_tag >= len(tags):
          return 'Done'
      for sent in doc.sents:
          tokens = []
          ner_tags_labels = []
          for token in sent.tokens:
              tokens.append(token.text)
              ner_tags_labels.append(token.tag)
          c = Counter(ner_tags_labels)
          count_tag = count_tag + c[b_tag] + c[i_tag]
          if c[b_tag] + c[i_tag] > 0:
              if {'tokens': tokens, 'ner_tags_labels': ner_tags_labels} not in result:
                  result.append({'tokens': tokens, 'ner_tags_labels': ner_tags_labels})

In [ ]:
result = []
all_tag = ['PER', 'LOC', 'ORG']
for tag in tqdm(all_tag):
    find_in_nerus(docs, tag)

100%|██████████| 3/3 [23:12<00:00, 464.00s/it]


In [ ]:
full_list = []
for dictionary in tqdm(tags):
    lil_list = []
    for idx, element in enumerate(dictionary['tokens']):
        lil_list.append({
            'text': element,
            'bio-tag': dictionary['ner_tags_labels'][idx]
        })
    full_list.append(lil_list)
for dictionary in tqdm(result):
    lil_list = []
    for idx, element in enumerate(dictionary['tokens']):
        lil_list.append({
            'text': element,
            'bio-tag': dictionary['ner_tags_labels'][idx]
        })
    full_list.append(lil_list)    

100%|██████████| 109723/109723 [00:02<00:00, 54068.79it/s]


In [ ]:
random.shuffle(full_list)

In [ ]:
train_sents, test_sents = train_test_split(full_list)

In [ ]:
train_sents[0]

[{'text': 'URBAN', 'bio-tag': 'B-BRAND'},
 {'text': 'ECO', 'bio-tag': 'I-BRAND'},
 {'text': 'SPIRYLINA', 'bio-tag': 'O'},
 {'text': '/', 'bio-tag': 'O'},
 {'text': 'Спирулина', 'bio-tag': 'O'},
 {'text': 'прессованная', 'bio-tag': 'O'},
 {'text': 'в', 'bio-tag': 'O'},
 {'text': 'таблетках', 'bio-tag': 'O'},
 {'text': '125гр', 'bio-tag': 'O'},
 {'text': '.', 'bio-tag': 'O'},
 {'text': '(', 'bio-tag': 'O'},
 {'text': '500таб', 'bio-tag': 'O'},
 {'text': '.', 'bio-tag': 'O'},
 {'text': 'по', 'bio-tag': 'O'},
 {'text': '250мг', 'bio-tag': 'O'},
 {'text': ')', 'bio-tag': 'O'}]

In [ ]:
test_sents[0]

[{'text': 'Об', 'bio-tag': 'O'},
 {'text': 'этом', 'bio-tag': 'O'},
 {'text': 'говорится', 'bio-tag': 'O'},
 {'text': 'в', 'bio-tag': 'O'},
 {'text': 'аналитических', 'bio-tag': 'O'},
 {'text': 'материалах', 'bio-tag': 'O'},
 {'text': '«', 'bio-tag': 'O'},
 {'text': 'НДВ-Супермаркет', 'bio-tag': 'B-ORG'},
 {'text': 'недвижимости', 'bio-tag': 'I-ORG'},
 {'text': '»', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'поступивших', 'bio-tag': 'O'},
 {'text': 'в', 'bio-tag': 'O'},
 {'text': '«', 'bio-tag': 'O'},
 {'text': 'Дом', 'bio-tag': 'B-ORG'},
 {'text': '»', 'bio-tag': 'O'},
 {'text': 'в', 'bio-tag': 'O'},
 {'text': 'четверг', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': '5', 'bio-tag': 'O'},
 {'text': 'апреля', 'bio-tag': 'O'},
 {'text': '.', 'bio-tag': 'O'}]

In [ ]:
def word2features(sent, i):
    word = sent[i]['text']
    
    features = {
        'text': word
    }
    if i > 0:
        word1 = sent[i-1]['text']
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1]['text']
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word['bio-tag'] for word in sent]

def sent2tokens(sent):
    return [word['text'] for word in sent]

In [ ]:
X_train = [sent2features(text) for text in train_sents]
y_train = [sent2labels(text) for text in train_sents]

X_test = [sent2features(text) for text in test_sents]
y_test = [sent2labels(text) for text in test_sents]

In [ ]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass
y_pred  = crf.predict(X_test)

In [ ]:
labels = set()
for y_big in y_train:
  for y_small in y_big:
    labels.add(y_small)

labels = list(labels)

In [ ]:
crf_metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.977937953366977

In [ ]:
crf_metrics.flat_precision_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9780859047365308

In [ ]:
crf_metrics.flat_recall_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9783640054334445